# Ajuste fino del modelo `tiny-llama`

**ESTE NOTEBOOK NO FUNCIONARÁ EN UN MAC OS POR LOS REQUISITOS TÉCNICOS DEL TRAINER**

In [3]:
from huggingface_hub import login
token = ""
print("Hugging Face logging")
login(token)

Hugging Face logging


In [4]:
import torch
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")


Usando dispositivo: cuda


In [ ]:
!pip install datasets trl

## Cargar el modelo + tokenizador

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
cache_dir = "./models/TinyLlama-1.1B-Chat-v1.0"

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float32 if device == "mps" else (torch.float16 if torch.cuda.is_available() else torch.float32),
            cache_dir=cache_dir,
            local_files_only=False,
        ).to(device)

model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Cargar el dataset y preprocesar los datos

In [9]:
import torch
from datasets import load_dataset

dataset_name = "mlabonne/guanaco-llama2-1k"
dataset = load_dataset(dataset_name, split="train", cache_dir="./data/guanaco-llama2-1k")

def preprocess_function(examples):
    max_length = 512  # Longitud máxima que permite el modelo
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=max_length,
    )
    return tokenized
dataset = dataset.map(preprocess_function, batched=True)


print(dataset)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 1000
})


## Configurar y ejecutar el entrenamiento

Configurar PEFT (ajuste fino eficiente de parámetros)

In [6]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import DefaultDataCollator
from transformers import DataCollatorForLanguageModeling


training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    max_steps=1000,
    learning_rate=2e-4,
    fp16=False, # Poner a true si hay cuda
    logging_steps=10,
    output_dir="./output",
    save_total_limit=3,
    report_to="none",  # Desactiva W&B
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Establece en False para modelos de lenguaje causal
)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    #peft_config=lora_config,
    data_collator=data_collator,
    args=training_args,
)
fine_tuned_model = "./models/TinyLlama-FT"
trainer.train()
trainer.save_model(fine_tuned_model)

Step,Training Loss
10,146.402700
20,0.000000
30,0.000000
40,0.000000
50,0.000000
60,0.000000
70,0.000000
80,0.000000
90,0.000000
100,0.000000


Step,Training Loss
10,146.402700
20,0.000000
30,0.000000
40,0.000000
50,0.000000
60,0.000000
70,0.000000
80,0.000000
90,0.000000
100,0.000000


# Uso de los modelos

In [11]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

class DialogueController:

    def __init__(self, system_prompt, assistant_token="<|assistant|>", keep_cronology=True):
        self.system_prompt = system_prompt
        self.prompt = [ { "role": "system", "content": system_prompt} ]
        self.assistant_token = assistant_token
        self.keep_cronology = keep_cronology

    def get_prompt(self):
        return self.prompt

    def add_user_prompt(self, user_prompt, role="user"):
        user_prompt_json = { "role": role, "content":  user_prompt}
        if self.keep_cronology:
          self.prompt.append(user_prompt_json)
        else:
          self.prompt = [self.prompt[0], user_prompt_json]
        return self.prompt

    def add_assistant_prompt(self, assistant_prompt):
        if self.keep_cronology:
          assistant_prompt_exclusive = assistant_prompt.split(self.assistant_token)
          final_prompt = ""
          if len(assistant_prompt_exclusive) == 2:
              final_prompt = assistant_prompt_exclusive[1]
          self.add_user_prompt(final_prompt, role="assistant")


class Chatbot2:

    def __init__(self, controller, chatbot_model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", cache_dir="./models/TinyLlama-1.1B-Chat-v1.0"):
        global device
        self.tokenizer = AutoTokenizer.from_pretrained(chatbot_model)
        self.model = AutoModelForCausalLM.from_pretrained(
            chatbot_model,
            torch_dtype=torch.float32 if device == "mps" else (torch.float16 if torch.cuda.is_available() else torch.float32),
            cache_dir=cache_dir,
            local_files_only=False,
        ).to(device)
        self.model_device = device
        self.dialogue_controller = controller

    def __run_prompt(self, prompt, do_sample, temperature, top_p, max_length, show_prompt):
        formatted_prompt = self.tokenizer.apply_chat_template(conversation=prompt, tokenize=False, return_dict=False, add_generation_prompt=True)

        # Tokenizar
        inputs = self.tokenizer(formatted_prompt, max_length=max_length, truncation=True, return_tensors="pt")
        inputs = {k: v.to(self.model_device) for k, v in inputs.items()}

        # Muestra infomacion de log
        if show_prompt:
          print(formatted_prompt)
          print("--- Token size: ---")
          [print("\t", k, ": ", len(v[0])) for k, v in inputs.items()]
          print("-------------------")

        # Generar respuesta
        outputs = self.model.generate(
            **inputs,
            temperature=temperature,
            top_p=top_p,
            do_sample=do_sample,
            pad_token_id=self.tokenizer.eos_token_id
        )

        # Decodificar y limpiar respuesta
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)


    def answer(self, user_prompt, do_sample=True, temperature=0.1, top_p=0.9, max_length=2047, show_prompt=False):
        # Actualiza el prompt
        prompt = self.dialogue_controller.add_user_prompt(user_prompt)
        # Resolver prompt
        answer = self.__run_prompt(prompt, do_sample, temperature, top_p, max_length, show_prompt)
        # Actualiza el prompt con la respuesta del asistente
        self.dialogue_controller.add_assistant_prompt(answer)
        return answer



In [14]:
# Modelos disponibles
fine_tuned_model = "./models/TinyLlama-FT"
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
cache_dir = "./models/TinyLlama-1.1B-Chat-v1.0"

# Cargar los modelos a través del chatbot
dialog_controller1 = DialogueController(system_prompt="Responde mis dudas de manera coincisa, breve, y siempre en español. En caso de no conocer la respuesta dime 'no lo se'.",keep_cronology=False)
dialog_controller2 = DialogueController(system_prompt="Responde mis dudas de manera coincisa, breve, y siempre en español. En caso de no conocer la respuesta dime 'no lo se'.",keep_cronology=False)

chatbot_finetuned = Chatbot2(controller=dialog_controller1, chatbot_model=model_name, cache_dir=fine_tuned_model)
chatbot_standard = Chatbot2(controller=dialog_controller2)

questions = [
    ("Cuál es la capital de francia?", "Paris"),
    ("¿Qué juegos me recomendarías si me ha gustado Undertale?","Deltarune, EarthBound, Hollow Knight, Hyper Light Drifter, Ori and the Blind Forest"),
    ("¡Hola! ¿Me podrías ayudar con un problema? No me encuentro del todo bien y me duele bastante la cabeza, ¿Sabes que podría ser?", "Puede que estés sufriendo de un dolor de cabeza. El dolor de cabeza puede ser causado por muchas cosas, desde estrés hasta una lesión. Si el dolor de cabeza persiste o empeora, es importante consultar a un médico para descartar cualquier afección grave. Algunas cosas que puedes hacer para aliviar el dolor de cabeza incluyen tomar analgésicos, evitar el estrés, descansar, practicar técnicas de relajación y evitar los desencadenantes como la cafeína, el alcohol y los cambios repentinos de presión."),
    ("quien era Francisco Morazán?", "Francisco Morazán fue un líder político y militar hondureño que trabajó para unir a los países de Centroamérica en una sola nación durante la primera mitad del siglo XIX. Fue elegido presidente de Honduras y de la Confederación Centroamericana y promovió la educación, la justicia y la igualdad. A pesar de ser derrocado y exiliado varias veces, Morazán es recordado como un héroe nacional en Honduras y en otros países de Centroamérica.")
]
for question, answer in questions:
    print("regular: ",chatbot_standard.answer(user_prompt=question))
    print("fine tuned: ",chatbot_finetuned.answer(user_prompt=question))
    print("****expected answer: ", answer)
    print("----------------------")



regular:  <|system|>
Responde mis dudas de manera coincisa, breve, y siempre en español. En caso de no conocer la respuesta dime 'no lo se'. 
<|user|>
Cuál es la capital de francia? 
<|assistant|>
La capital de Francia es París.
fine tuned:  <|system|>
Responde mis dudas de manera coincisa, breve, y siempre en español. En caso de no conocer la respuesta dime 'no lo se'. 
<|user|>
Cuál es la capital de francia? 
<|assistant|>
La capital de Francia es París.
****expected answer:  Paris
----------------------
regular:  <|system|>
Responde mis dudas de manera coincisa, breve, y siempre en español. En caso de no conocer la respuesta dime 'no lo se'. 
<|user|>
¿Qué juegos me recomendarías si me ha gustado Undertale? 
<|assistant|>
Sure, Undertale is a great game to play if you enjoyed the aesthetic and gameplay of Toby Fox's Undertale. Here are some games that you might enjoy:

1. Celeste: This game is a platformer with a unique twist - the gameplay is based on the player's breathing. It's a